# Jar Targetting System
## Training Program
Liam Akkerman, Aidan Hunter

In [ ]:
# load libraries

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

In [ ]:
# TODO load the dataset

train_images = 
train_labels = 

test_images = 
test_labels = 

In [ ]:
# TODO visualize samples

In [ ]:
# define the model layers
# TODO currently this is just the 4th CNN from assigment 3

model = keras.Sequential([
    keras.layers.Input(shape=(300, 300, 1)),
    keras.layers.AveragePooling2D(pool_size=6, strides=3),
    keras.layers.Conv2D(filters=1, kernel_size=3, activation='relu'),
    keras.layers.Conv2D(filters=1, kernel_size=3, activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(rate=0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(units=8, activation='softmax')
])

In [ ]:
# train and evalute the model

model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

hist = model.fit(x=train_images, y=train_labels, epochs=10)
loss, acc = model.evaluate(test_images, test_labels)

print('test accuracy:', acc)
print('test loss:', loss)

In [4]:
# save model

from datetime import datetime

# format is YYYYMMDDHHmm
filename = 'model-' + datetime.now().strftime('%Y%m%d%H%M')

model.save(filename)

model-202103201625
